# AI+ 1-step Prediction Pipeline

This notebook will accept a folder of new ultrasound images and labels
    and return a dataframe of results and a dataframe of MAP scores

To use this notebook:

0.  Install relevant packages including ultralytics, tqdm, mapcalc
1.  Prepare images by cropping border artifacts/text, leaving only the full ultrasound image
    : See the notebook \data_preprocessing\image_cropping_AIplus.ipynb for example of how to do this
2.  Place images in the local subdirectory \Data\ 
    following the pattern \Data\Test_images\imgs and \Data\Test_images\jsons
3.  Place trained model in the \Trained_Models\ subdirectory
4.  Update the model and data paths in the cell below this one
5.  Run the notebook cells in sequence

In [10]:
import os
this_path = os.getcwd()
data_basepath = this_path + '\\Data\\External_test_cropped\\'
model_basepath = this_path + '\\Trained_Models\\'

# Update this with path to trained YOLOv8 1-step model
MODEL_PATH = model_basepath + 'best_yolo_1step.pt'

# Update this with path to images and labels to be evaluated
TEST_SET_PATH = data_basepath

Import relevant functions

In [11]:
import cv2
import sys
import json

from tqdm import tqdm
import pandas as pd

import ultralytics

from ultralytics import YOLO
from matplotlib import pyplot as plt

from glob import glob
import numpy as np
from mapcalc import calculate_map, calculate_map_range

ultralytics.checks()

Ultralytics YOLOv8.0.142  Python-3.10.2 torch-2.0.1+cpu CPU (11th Gen Intel Core(TM) i7-1185G7 3.00GHz)
Setup complete  (8 CPUs, 15.7 GB RAM, 409.3/952.6 GB disk)


Function to draw images with annotations

In [3]:
def draw_boxes(image_path, boxes, labels, scores, ground_truth_boxes, ground_truth_labels, output_path, normalized_bbox=True):
    # Read the image
    image = cv2.imread(image_path)

    # Get the image dimensions
    height, width = image.shape[:2]

    # Iterate over the boxes
    for (box, label, score) in zip(boxes, labels, scores):
        # Convert normalized coordinates to pixel coordinates
        x1, y1, x2, y2 = tuple(box)
        if normalized_bbox:
            x1, y1, x2, y2 = int(x1 * width), int(y1 * height), int(x2 * width), int(y2 * height)
        else:
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

        # Draw the box

        # Multi-class
        if label == 0:
            tumor = "benign"
            color = (0, 255, 0)
        elif label == 1:
            tumor = "malignant"
            color = (0, 0, 255)
        
        # # Single-class
        # if label == 0:
        #     tumor = "tumor"
        #     color = (255, 0, 0)

        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

        # Prepare the label and confidence
        text = f"{tumor}: {score:.2f}"

        # Draw the label and confidence
        cv2.putText(image, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

    for (box, label) in zip(ground_truth_boxes, ground_truth_labels):
        # Convert normalized coordinates to pixel coordinates
        x1, y1, x2, y2 = tuple(box)
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

        # Draw the box

        if label == 0:
            tumor = "I-benign"
            color = (0, 255, 0)
        elif label == 1:
            tumor = "I-malignant"
            color = (0, 0, 255)

        # Draw the label and confidence
        cv2.putText(image, tumor, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

        print(image_path)
        print(label)

        cv2.rectangle(image, (x1, y1), (x2, y2), color, 1)

    # Save the image
    cv2.imwrite(output_path, image)

In [12]:
# set relevant paths
imgs_dir = os.path.join(TEST_SET_PATH, "imgs")
output_dir = os.path.join(TEST_SET_PATH, "outputs")
jsons_dir= os.path.join(TEST_SET_PATH, "jsons")
img_list = glob(os.path.join(imgs_dir, "*.jpg"))

print(f"Total Number of Test Images: {len(img_list)}")

Total Number of Test Images: 4


In [13]:
# Load Trained Model
model = YOLO(MODEL_PATH)

Run Prediction

In [14]:

map_dict = {"img_name":[], "mAP50":[], "mAP50-95":[]}

for img in tqdm(img_list):

    result = model.predict(img, imgsz=640, conf=0.5, iou=0.7, augment=True)
    # result = model.predict(img, imgsz=640, conf=0.25, iou=0.7, augment=True)

    img_name = os.path.basename(img)
    output_path = os.path.join(output_dir, img_name)
    json_file = os.path.join(jsons_dir, img_name.replace(".jpg", ".json"))

    with open(json_file, "r") as f:
        test_data = json.load(f)

    ground_truth = {"boxes":[], "labels":[]}
    result_dict = {"boxes":[], "labels":[], "scores":[]}

    for shape in test_data["shapes"]:
        # if shape['label'] != "border":
        #     ground_truth["labels"].append(shape["label"])
        #     ground_truth["boxes"].append([shape["points"][0][0], shape["points"][0][1], shape["points"][1][0], shape["points"][1][1]])   

        # Multi-class
        if shape['label'] == "benign":
            shape['label'] = 0
            ground_truth["labels"].append(shape["label"])
            ground_truth["boxes"].append([shape["points"][0][0], shape["points"][0][1], shape["points"][1][0], shape["points"][1][1]])            
        elif shape['label'] == "malignant":
            shape['label'] = 1
            ground_truth["labels"].append(shape["label"])
            ground_truth["boxes"].append([shape["points"][0][0], shape["points"][0][1], shape["points"][1][0], shape["points"][1][1]])
        
        # if shape['label'] != 'border':
        #     shape['label'] = 0
        #     ground_truth["labels"].append(shape["label"])
        #     ground_truth["boxes"].append([shape["points"][0][0], shape["points"][0][1], shape["points"][1][0], shape["points"][1][1]])

    cls_list = []
    conf_list = []
    nboxes_list = []
    boxes_list = []

    for i in range(len(result[0].boxes.cls.cpu().detach().numpy())):
        cls_list.append(int(result[0].boxes.cls.cpu().detach().numpy()[i].tolist()))
        conf_list.append(result[0].boxes.conf.cpu().detach().numpy()[i].tolist())
        nboxes_list.append(result[0].boxes.xyxyn.cpu().detach().numpy()[i].tolist())
        boxes_list.append(result[0].boxes.xyxy.cpu().detach().numpy()[i].tolist())

        result_dict["labels"].append(int(result[0].boxes.cls.cpu().detach().numpy()[i].tolist()))
        result_dict["boxes"].append(result[0].boxes.xyxy.cpu().detach().numpy()[i].tolist())
        result_dict["scores"].append(result[0].boxes.conf.cpu().detach().numpy()[i].tolist())
    
    draw_boxes(img, nboxes_list, cls_list, conf_list, ground_truth['boxes'], ground_truth['labels'], output_path)

    # print(ground_truth)
    # print(result_dict)

    map50 = calculate_map(ground_truth, result_dict, 0.5)
    map5095 = calculate_map_range(ground_truth, result_dict, 0.5, 0.95, 0.05)

    map_dict["img_name"].append(img_name)
    map_dict["mAP50"].append(map50)
    map_dict["mAP50-95"].append(map5095)

  0%|          | 0/4 [00:00<?, ?it/s]


image 1/1 c:\GTMS Projects\Practicum\AIplus-Practicum-Final\Data\External_test_cropped\imgs\0AD131A0D42742098EE0ADD670C4C5C2_2737480.jpg: 448x640 1 malignant, 1770.2ms
Speed: 2.0ms preprocess, 1770.2ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)
 25%|██▌       | 1/4 [00:03<00:11,  3.75s/it]


c:\GTMS Projects\Practicum\AIplus-Practicum-Final\Data\External_test_cropped\imgs\0AD131A0D42742098EE0ADD670C4C5C2_2737480.jpg
1


image 1/1 c:\GTMS Projects\Practicum\AIplus-Practicum-Final\Data\External_test_cropped\imgs\0AD131A0D42742098EE0ADD670C4C5C2_2737481.jpg: 480x640 1 malignant, 1922.0ms
Speed: 2.0ms preprocess, 1922.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
 50%|█████     | 2/4 [00:05<00:05,  2.70s/it]


c:\GTMS Projects\Practicum\AIplus-Practicum-Final\Data\External_test_cropped\imgs\0AD131A0D42742098EE0ADD670C4C5C2_2737481.jpg
1


image 1/1 c:\GTMS Projects\Practicum\AIplus-Practicum-Final\Data\External_test_cropped\imgs\0AD131A0D42742098EE0ADD670C4C5C2_2737487.jpg: 576x640 2 malignants, 2264.3ms
Speed: 3.1ms preprocess, 2264.3ms inference, 1.0ms postprocess per image at shape (1, 3, 576, 640)
 75%|███████▌  | 3/4 [00:08<00:02,  2.51s/it]


c:\GTMS Projects\Practicum\AIplus-Practicum-Final\Data\External_test_cropped\imgs\0AD131A0D42742098EE0ADD670C4C5C2_2737487.jpg
1


image 1/1 c:\GTMS Projects\Practicum\AIplus-Practicum-Final\Data\External_test_cropped\imgs\0AD131A0D42742098EE0ADD670C4C5C2_2737488.jpg: 576x640 1 malignant, 2378.5ms
Speed: 4.0ms preprocess, 2378.5ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)
100%|██████████| 4/4 [00:10<00:00,  2.61s/it]

c:\GTMS Projects\Practicum\AIplus-Practicum-Final\Data\External_test_cropped\imgs\0AD131A0D42742098EE0ADD670C4C5C2_2737488.jpg
1


In [15]:
df = pd.DataFrame(map_dict)
print(df.iloc[0:5,])

                                       img_name  mAP50  mAP50-95
0  0AD131A0D42742098EE0ADD670C4C5C2_2737480.jpg    1.0       0.1
1  0AD131A0D42742098EE0ADD670C4C5C2_2737481.jpg    1.0       0.6
2  0AD131A0D42742098EE0ADD670C4C5C2_2737487.jpg    1.0       0.4
3  0AD131A0D42742098EE0ADD670C4C5C2_2737488.jpg    1.0       0.5


Display Accuracy metrics

In [16]:
# mAP50
print('mAP50 score:',round(df['mAP50'].mean(),3))
print('mAP50-95 score:',round(df['mAP50-95'].mean(),3))
# Images Not detected Correctly
print('# of images not detected correctly:',len(df[df['mAP50']==0]))

mAP50 score: 1.0
mAP50-95 score: 0.4
# of images not detected correctly: 0


In [9]:
# Save mAP results to csv file
df.to_csv(os.path.join(TEST_SET_PATH, "mAP_results.csv"))